In [1]:
import psycopg2
import pandas as pd
import numpy as np
from configparser import ConfigParser
from sqlalchemy import create_engine, engine_from_config
from sqlalchemy.engine import URL

# Our libraries.
from config import config
from connect import connect
from query import query
from create_table import table_from_df
from get_postcode_mapping_table import get_postcode_mapping_table

In [2]:
# currently using dummy excel files to read in as DataFrames

listings_df = pd.read_excel(r'listings_test.xlsx')
sold_df = pd.read_excel(r'sold_test.xlsx')

postcode_df = get_postcode_mapping_table()

C:\Users\gatkinson\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3400: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


In [3]:
# create new database tables from listings and sold DataFrames.

table_from_df(listings_df, 'listings')

Connecting to the PostgreSQL database...
Table listings created
Database connection closed.


In [4]:
table_from_df(sold_df, 'sold')

Connecting to the PostgreSQL database...
Table sold created
Database connection closed.


In [5]:
table_from_df(postcode_df, 'postcodes')

Connecting to the PostgreSQL database...
Table postcodes created
Database connection closed.


In [10]:
# Join location data to sold and listings tables from postcodes table.

query('ALTER TABLE listings ADD COLUMN district VARCHAR(100);','listings', output = None)


Section postgresql not found in the config.ini file


In [7]:
query('UPDATE listings SET district = ladnm FROM postcodes WHERE listings.address = postcodes.pcds;','listings', output = None)


Connecting to the PostgreSQL database...
Query executed.
no results to fetch
Database connection closed.


In [8]:
query('ALTER TABLE sold ADD COLUMN district VARCHAR(100);','sold', output = None)


Connecting to the PostgreSQL database...
Query executed.
no results to fetch
Database connection closed.


In [9]:
query('UPDATE sold SET district = ladnm FROM postcodes WHERE sold.address = postcodes.pcds;','sold', output = None)

Connecting to the PostgreSQL database...
Query executed.
no results to fetch
Database connection closed.


In [ ]:
# Extract sold table to train ML model on

sold_export = query('SELECT * FROM sold', 'sold', output = 'df')

In [ ]:
sold_export

In [ ]:
# Extract listings table to make predictions on

listings_export = query('SELECT * FROM listings', 'listings', output = 'df')

In [ ]:
listings_export

In [ ]:
# Imagine that the export has been put through the ML Pipeline and a prediction has been made.

listings_export['Prediction'] = np.random.normal(listings_export['Listed Price'], 0.1*listings_export['Listed Price']).round(2)

In [ ]:
listings_export['Value Delta'] = (listings_export['Prediction'] - listings_export['Listed Price']).round(2)

In [ ]:
listings_export_trim = listings_export[['ID', 'Prediction', 'Value Delta']]

In [ ]:
# Feed predictions back into database as a new table.

table_from_df(listings_export_trim,'predictions')